# Agentic Search 

<p>Start agentic search using Langchain. We will define a agent with the following tools:</p>

- Ability to query our Qdrant database 
  
  - Allow it to also make it's own queries and get the best query
  
- Ability to open our pdf and read a particular page

In [46]:
# !pip install -qU langchain-qdrant

In [47]:
MAX_CHUNK_SIZE = 1024

In [48]:
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_qdrant import QdrantVectorStore
from langchain.tools.retriever import create_retriever_tool
from qdrant_client import QdrantClient
from langchain_core.tools import Tool
from langchain_core.tools import tool
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)

from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI 

embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small", chunk_size=MAX_CHUNK_SIZE)
client = QdrantClient(url="http://localhost:6333")

In [49]:
vector_store = QdrantVectorStore(
    client=client,
    collection_name="pdf_collection",
    embedding=embeddings_model,
)
retriever = vector_store.as_retriever()

## Building The Agent

In [50]:
@tool
def natural_language_query(input: str):
    """ Will query the vector store that contains information about the research papers and return the top 10 most relevant chunks from all the papers.
    Each result has the following structure:
    {
        "text": "string",
        "pdf_path": "string", # the path to the pdf file
        "page_number": 0, # the page number where the text is located
        "idx": string # the pdf is chunked into smaller pieces, this is the id of the chunk. eg: 1.txt, 2.txt, etc. Each document is split sequentially into chunks.
        "score": 0.0 # the cosine similarity between the input and the text
    }
    """
    result = client.search(collection_name="pdf_collection", query_vector=embeddings_model.embed_documents([input])[0], limit=10)
    return [{'text': r.payload['text'], 'pdf_path': r.payload['pdf_file'], 'page_number': r.payload['page'], 'idx': r.payload['idx'], 'score': r.score} for r in result]

In [51]:
research_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        """
        You are a agent that is going to be searching technical research papers. You will be given a topic to find information on. You will need to find the most relevant information and return back the information with sources. 
        Make sure you only return information that you have retrieved from the research papers. Do not make up any information. You will have access to tools to help you find the information.

        Make sure to make uses of the tools provided. You can use the tools as many times as needed. Make sure you have all the information needed before returning the information.

        Return the output by stating the information you found and the source of the information in the format pdf_path - page number - the section (if mentioned) in the papers we queried. The page number and pdf_path should only be from our vector store, do not make up any page numbers or pdf paths.

        eg: 
        // brief summary of the information only based on the tools we used
        // pdf_path 
        // page number
        // section (if it can be found in the text of the result)
        """
    ),
    ("human", "Research {research_topic}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [52]:
llm = ChatOpenAI(
    model="gpt-4o",
    max_retries=2,
    temperature=0.1,
)
agent_tools = [natural_language_query]
llm = llm.bind_tools(agent_tools)

In [53]:
def agent_scratchpad_formatter(x):
    return format_to_openai_tool_messages(x["intermediate_steps"])

agent = (
    {
        "research_topic": lambda x: x["research_topic"],
        "agent_scratchpad": agent_scratchpad_formatter,
    }
    | research_prompt
    | llm
    | OpenAIToolsAgentOutputParser() # This gets the output from the OpenAI api call and gets the tool invocations from it. This then is executed by the AgentExecutor.
)

In [54]:
agent_executor = AgentExecutor(agent=agent, tools=agent_tools, verbose=True)

In [ ]:
research_agent_result = await agent_executor.ainvoke({"research_topic": "Where is the data from the map function stored?"})
research_agent_result['output']



> Entering new AgentExecutor chain...

Invoking: `natural_language_query` with `{'input': 'Where is the data from the map function stored?'}`


[{'text': 'The master is the conduit through which the location of interme- diate file regions is propagated from map tasks to reduce tasks. There - fore, for each completed map task, the master stores the locations and sizes of the R intermediate file regions produced by the map task. Updates to this location and size information are received as map tasks are completed. The information is pushed incrementally to workers that have in-progress reduce tasks.', 'pdf_path': '/Users/faizahmed/Documents/SJSU/topics_in_db/Project/data-ingestion/data/mapreduce.pdf', 'page_number': '3', 'idx': '15.txt', 'score': 0.41727197}, {'text': '2. One of the copies of the program—the master— is special. The rest are workers that are assigned work by the master. There are M map tasks and R reduce tasks to assign. The master picks idle workers and assigns each on

{'research_topic': 'Where is the data from the map function stored?',
 'output': 'The data from the map function is initially buffered in memory and then periodically written to the local disk. It is partitioned into regions by a partitioning function, and the locations of these buffered pairs on the local disk are passed back to the master. The master is responsible for forwarding these locations to the reduce workers. \n\n- **pdf_path**: /Users/faizahmed/Documents/SJSU/topics_in_db/Project/data-ingestion/data/mapreduce.pdf\n- **page number**: 2\n\nAdditionally, for each completed map task, the master stores the locations and sizes of the intermediate file regions produced by the map task. This information is pushed incrementally to workers that have in-progress reduce tasks.\n\n- **pdf_path**: /Users/faizahmed/Documents/SJSU/topics_in_db/Project/data-ingestion/data/mapreduce.pdf\n- **page number**: 3'}